In [1]:
import pandas as pd
import numpy as np
import sys
from scipy import sparse
#0 is good, 1 is bad.
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
#reading in data
df = pd.read_csv('../data/priors_sample.csv')

In [3]:
df.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,order_id,add_to_cart_order,reordered,user_id,eval_set,...,average_orders,average_order_dow,average_order_hour,weekend,rate_reordered,department,aisle,aisle_cat,dept_cat,product_cat
0,2,1,Chocolate Sandwich Cookies,61,19,9228,2,0.0,79603,prior,...,0.0,0.0,0.0,0,0.000000,snacks,cookies cakes,0,0,NaN
1,22,1,Chocolate Sandwich Cookies,61,19,47122,6,1.0,38748,prior,...,0.0,0.0,0.0,1,0.714286,snacks,cookies cakes,0,0,NaN
2,29,1,Chocolate Sandwich Cookies,61,19,56811,1,1.0,113429,prior,...,0.0,0.0,0.0,0,1.000000,snacks,cookies cakes,0,0,NaN
3,30,1,Chocolate Sandwich Cookies,61,19,57503,3,1.0,111982,prior,...,0.0,0.0,0.0,1,0.000000,snacks,cookies cakes,0,0,NaN
4,31,1,Chocolate Sandwich Cookies,61,19,58691,14,1.0,144159,prior,...,0.0,0.0,0.0,0,0.000000,snacks,cookies cakes,0,0,NaN


# Filter data by reorders

for users that have re-ordered, what should they order next based on their order history

In [4]:
#Reorders only
reordered = df[df.reordered == 1]
reordered.head(1)

,Unnamed: 0,product_id,product_name,aisle_id,department_id,order_id,add_to_cart_order,reordered,user_id,eval_set,...,average_orders,average_order_dow,average_order_hour,weekend,rate_reordered,department,aisle,aisle_cat,dept_cat,product_cat
1,22,1,Chocolate Sandwich Cookies,61,19,47122,6,1.0,38748,prior,...,0.0,0.0,0.0,1,0.714286,snacks,cookies cakes,0,0,NaN


In [5]:
reordered.drop(columns = ['Unnamed: 0'], inplace = True)

/Users/maithilijoshi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [6]:
reordered.describe()

,product_id,aisle_id,department_id,order_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,average_orders,average_order_dow,average_order_hour,weekend,rate_reordered,aisle_cat,dept_cat,product_cat
count,334259.000000,334259.000000,334259.000000,334259.000000,334259.000000,334259.0,334259.000000,334259.000000,334259.000000,334259.000000,334259.000000,334259.000000,334259.000000,334259.000000,334259.000000,334259.000000,334259.000000,334259.000000,292323.000000
mean,25571.394706,71.365154,9.682689,244656.747462,7.552640,1.0,79091.504725,21.823218,2.745592,13.363156,10.178472,0.926680,0.189272,0.931990,0.366659,0.057958,52.830449,7.865927,6.871348
std,14000.305906,38.208517,6.316908,141315.018678,6.586228,0.0,46164.670603,18.833484,2.095275,4.262116,8.078147,3.716589,0.736376,3.476074,0.481893,0.205792,34.942240,5.197273,5.144395
min,1.000000,1.000000,1.000000,12.000000,1.000000,1.0,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13575.000000,31.000000,4.000000,122286.000000,3.000000,1.0,38808.000000,8.000000,1.000000,10.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,3.000000,3.000000
50%,24964.000000,83.000000,7.000000,244582.000000,6.000000,1.0,79259.000000,16.000000,3.000000,13.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,48.000000,8.000000,5.000000
75%,37687.000000,107.000000,16.000000,366777.000000,10.000000,1.0,118542.000000,30.000000,5.000000,16.000000,13.000000,0.000000,0.000000,0.000000,1.000000,0.000000,75.000000,12.000000,13.000000
max,49683.000000,134.000000,21.000000,490949.000000,70.000000,1.0,159351.000000,99.000000,6.000000,23.000000,30.000000,26.545455,6.000000,21.000000,1.000000,1.000000,133.000000,20.000000,17.000000


In [7]:
pivot = reordered.pivot_table(index = 'product_name', 
                                columns = 'product_id', 
                                values = 'reordered')

pivot.head()

product_id,1,3,4,8,10,11,12,23,25,26,...,49667,49668,49671,49674,49675,49678,49679,49680,49682,49683
product_name,,,,,,,,,,,,,,,,,,,,,
#2 Coffee Filters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#4 Natural Brown Coffee Filters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
& Go! Hazelnut Spread + Pretzel Sticks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(70% Juice!) Mountain Raspberry Juice Squeeze,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0% Fat Black Cherry Greek Yogurt y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pivot_sparce = sparse.csr_matrix(pivot.fillna(0))

In [9]:
recommend = pairwise_distances(pivot_sparce, metric = 'cosine')
recommend

array([[0., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.]])

In [10]:
recommend_df = pd.DataFrame(recommend, columns = pivot.index,
                           index = pivot.index)

recommend_df.head()

product_name,#2 Coffee Filters,#4 Natural Brown Coffee Filters,& Go! Hazelnut Spread + Pretzel Sticks,(70% Juice!) Mountain Raspberry Juice Squeeze,0% Fat Black Cherry Greek Yogurt y,0% Fat Blueberry Greek Yogurt,0% Fat Free Organic Milk,0% Fat Greek Yogurt Black Cherry on the Bottom,0% Fat Organic Greek Vanilla Yogurt,0% Fat Superfruits Greek Yogurt,...,vitaminwater® XXX Acai Blueberry Pomegranate,with Crispy Almonds Cereal,with Lime Juice Mayonesa Mayonnaise,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Pineapple Coconut Water
product_name,,,,,,,,,,,,,,,,,,,,,
#2 Coffee Filters,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
#4 Natural Brown Coffee Filters,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
& Go! Hazelnut Spread + Pretzel Sticks,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
(70% Juice!) Mountain Raspberry Juice Squeeze,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0% Fat Black Cherry Greek Yogurt y,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [11]:
reordered.product_name[1500:2000]

2437                                                 Soda
2438                                                 Soda
2439                                                 Soda
2440                                                 Soda
2441                                                 Soda
2442                                                 Soda
2443                                                 Soda
2446                                                 Soda
2447                                                 Soda
2448                                                 Soda
2455    Free & Clear Natural Laundry Detergent For Sen...
2457    Free & Clear Natural Laundry Detergent For Sen...
2458    Free & Clear Natural Laundry Detergent For Sen...
2459    Free & Clear Natural Laundry Detergent For Sen...
2461    Free & Clear Natural Laundry Detergent For Sen...
2464    Free & Clear Natural Laundry Detergent For Sen...
2466    Free & Clear Natural Laundry Detergent For Sen...
2469    Free &

In [13]:
recommend_df['Peaches'].sort_values()[1:11]

product_name
#2 Coffee Filters                                      1.0
Original Air Effects With Gain Air Freshening Spray    1.0
Original Acai Juice                                    1.0
Original 7\" Pizza Crusts                              1.0
Original 7 Grain Sea Salt Pita Crisps                  1.0
Original 7 Grain Crackers                              1.0
Original 5-Cheese Pizza                                1.0
Original 120 count Fabric Enhancers Dryer Sheets       1.0
Original 100% Vegetable Juice                          1.0
Original 100% Pure No Pulp Orange Juice                1.0
Name: Peaches, dtype: float64